SELECT winner, loser, __updatetime from CONTESTS 
WHERE game='madden19'
AND type='H'
AND status='completed'
LIMIT 500;

In [242]:
import matplotlib.pyplot as plt
%matplotlib inline

def moving_average(array, window=3):
    """
    a : array
    n : int, size of window
    """
    ret = np.cumsum(array, dtype=float)
    ret[window:] = ret[window:] - ret[:-window]
    return ret[window - 1:] / window



def _update_counter(player):
    if player in counter.keys():
        counter[player] += 1
    else:
        counter[player] = 1
        
class Player(object):
    def __init__(self, name):
        self.name = name
        self.ranking = Rating(50)  # everyone starts off with the mean rating
        self.count = 0
        
BETA = 8.333

def win_probability(greater, lesser):
    # # https://github.com/sublee/trueskill/issues/1
    exp = (greater.mu - lesser.mu) / BETA
    n = 4. ** exp
    return n / (n + 1)

from trueskill import Rating, quality_1vs1, rate_1vs1
import numpy as np
from tqdm import tqdm



def population2hist():
    count = {}
    for k,v in population.items():
        if k not in count.keys():
            count[k] = 1
        else:
            count[k] += 1

    histogram = {}
    for k, v in count.items():
        if v in histogram.keys():
            histogram[v] += 1
        else:
            histogram[v] = 1
    return histogram

def idx2player(idx):
    if idx not in population:
        population[idx] = Player(idx)
    return population[idx]

In [5]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine

In [6]:
POSTGRES_ADDRESS = 'db.panoply.io' ## INSERT YOUR DB ADDRESS IF IT'S NOT ON PANOPLY
POSTGRES_PORT = '5439'
POSTGRES_USERNAME = 'vin.tang@gmail.com' ## CHANGE THIS TO YOURPANOPLY/POSTGRES USERNAME
POSTGRES_PASSWORD = 'Cobalt27!' ## CHANGE THIS TO YOUR PANOPLY/POSTGRES PASSWORD
POSTGRES_DBNAME = 'playerslounge_prod' ## CHANGE THIS TO YOUR DATABASE NAME

In [7]:
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(username=POSTGRES_USERNAME,
password=POSTGRES_PASSWORD,
ipaddress=POSTGRES_ADDRESS,
port=POSTGRES_PORT,
dbname=POSTGRES_DBNAME))

In [8]:
cnx = create_engine(postgres_str)

In [280]:
query = """SELECT winner, loser, __updatetime from CONTESTS 
WHERE game='madden19' and type='H' and status='completed'
ORDER by __updatetime ASC
LIMIT 1000;"""

In [281]:
df = pd.read_sql_query(query, cnx)
df.shape

(1000, 3)

In [282]:
df.head()

,winner,loser,__updatetime
0,3458308436715043,4697239373145551,2019-01-14 18:58:16.776
1,3594214647178581,7804484557944950,2019-01-14 18:58:16.776
2,2140153741499499,6082031424054056,2019-01-14 18:58:16.776
3,4034393453488033,3619807442576769,2019-01-14 18:58:16.776
4,3594214647178581,4446065660420329,2019-01-14 18:58:16.776


In [286]:
population = {}
history = []

for i, row in tqdm(df.iterrows()):
    
    # check if winner exists and return P
    p1 = idx2player(row['winner'])
    p2 = idx2player(row['loser'])
    
    p1.count += 1
    p2.count += 1
            
    # generate prediction based on trueskill - where we correct?
    min_matches = 2
    if (p1.count > min_matches) and (p2.count > min_matches):
        prob = win_probability(p1.ranking, p2.ranking)  # winning probability for p1
        pred = np.random.uniform() < prob
        history.append(int(pred))
    
    # update players 
    p1.ranking, p2.ranking = rate_1vs1(p1.ranking, p2.ranking)

1000it [00:00, 1171.41it/s]


In [284]:
d = moving_average(history, window=int(len(d)/10))
plt.plot(d)
plt.ylim(.4, .7)

ValueError: operands could not be broadcast together with shapes (690,) (0,) 